In [6]:
import requests
import json
import pandas as pd

In [3]:
# loop through the pages in order to gather all the entries in a single list
def get_results(data, key):
    result = []
    current_page = data
    has_next = True
    
    while(has_next):
        for data_point in current_page['results']:
            result.append(data_point)

        if current_page['next'] == None:
            has_next = False
        else:
            response = requests.get(current_page['next'])
            current_page = response.json()

    return result

In [79]:
# get the json from the API
response = requests.get("https://swapi.dev/api/?format=json")
root = response.json()

data = {}

# get json from the major columns (people, planets, etc.)
for key in root:
    response = requests.get(root[key] + '/?format=json')
    data[key] = get_results(response.json(), key)
    print(key, 'done')


people done
planets done
films done
species done
vehicles done
starships done


In [80]:
# get the names or titles instead of the url in lists within the nested objects
for v in data.values():
    for item in v:
        for key, value in item.items():
            original_key = key

            if key == 'pilots' or key == 'characters' or key == 'residents':
                original_key = 'people'

            if type(value) == list and original_key in data.keys():
                items = []
                for val in value:
                    for root_data in data[original_key]:
                        if root_data['url'] == val:
                            if ('name' in root_data):
                                items.append(root_data['name'])
                            else:
                                items.append(root_data['title'])

                            break
                item[key] = items

            if key == 'homeworld':
                original_key = 'planets'
                for root_data in data[original_key]:
                    if root_data['url'] == value:
                        item[key] = root_data['name']
                        break

# write finished json to file
with open("result.json", "w") as write_file:
    json.dump(data, write_file, indent=4)

In [87]:
# descriptive analysis for each column
for col in data:
    # create dataframe for each key (people, planets, etc.)
    nested_df = pd.json_normalize(data, col)

    print('\n#############################\n######  Column:', col, '\n#############################')
    print('\nColumns:')
    display(list(nested_df.columns))
    print('------------------\n\nHead (First 5 Items):')
    display(nested_df.head())
    print('------------------\n\nInfo:\n')
    display(nested_df.info())
    print('------------------\n\nDescription:')
    display(nested_df.describe())
    print('------------------\n\nCount values:')
    display(nested_df.count())


#############################
######  Column: people 
#############################

Columns:


['name',
 'height',
 'mass',
 'hair_color',
 'skin_color',
 'eye_color',
 'birth_year',
 'gender',
 'homeworld',
 'films',
 'species',
 'vehicles',
 'starships',
 'created',
 'edited',
 'url']

------------------

Head (First 5 Items):


,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,"[A New Hope, The Empire Strikes Back, Return o...",[],"[Snowspeeder, Imperial Speeder Bike]","[X-wing, Imperial shuttle]",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,Tatooine,"[A New Hope, The Empire Strikes Back, Return o...",[Droid],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,Naboo,"[A New Hope, The Empire Strikes Back, Return o...",[Droid],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,"[A New Hope, The Empire Strikes Back, Return o...",[],[],[TIE Advanced x1],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,"[A New Hope, The Empire Strikes Back, Return o...",[],[Imperial Speeder Bike],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/


------------------

Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        82 non-null     object
 1   height      82 non-null     object
 2   mass        82 non-null     object
 3   hair_color  82 non-null     object
 4   skin_color  82 non-null     object
 5   eye_color   82 non-null     object
 6   birth_year  82 non-null     object
 7   gender      82 non-null     object
 8   homeworld   82 non-null     object
 9   films       82 non-null     object
 10  species     82 non-null     object
 11  vehicles    82 non-null     object
 12  starships   82 non-null     object
 13  created     82 non-null     object
 14  edited      82 non-null     object
 15  url         82 non-null     object
dtypes: object(16)
memory usage: 10.4+ KB


...

None

------------------

Description:


,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
count,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36
unique,36,36,27,26,36,21,20,20,26,17,12,13,26,15,12,36,36,36
top,CR90 corvette,CR90 corvette,Kuat Systems Engineering,unknown,150,1050,1,0,unknown,2 years,1.0,unknown,Starfighter,[],[Revenge of the Sith],2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/
freq,1,1,4,10,1,5,11,11,7,6,15,19,5,21,9,1,1,1


------------------

Count values:


name                      36
model                     36
manufacturer              36
cost_in_credits           36
length                    36
max_atmosphering_speed    36
crew                      36
passengers                36
cargo_capacity            36
consumables               36
hyperdrive_rating         36
MGLT                      36
starship_class            36
pilots                    36
films                     36
created                   36
edited                    36
url                       36
dtype: int64